# Numpy: manipulating arrays

Numpy's speedup is no joke.

**Normal Python:**

In [1]:
import random
data = []
for i in range(1000000):
    data.append(random.gauss(0, 1))

In [2]:
%%timeit
data2 = []
for x in data:
    data2.append(x**2)

201 ms ± 4.59 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Numpy:**

In [3]:
import numpy
data = numpy.random.normal(0, 1, 1000000)

In [4]:
%%timeit
data2 = data**2

919 µs ± 7.84 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


**A Numpy array is everything normal Python data is not:**

   * Loop performed in native bytecode (compiled C)
   * Type-checking performed once before loop
   * Data are packed in contiguous bytes
   * Python's Global Interpreter Lock (GIL) is released during loop

**Bonus:**

   * Most methods also benefit from hardware vectorization

But you have to write your algorithms "sideways."

In [5]:
px = numpy.random.normal(0, 30, 100000)
py = numpy.random.normal(0, 30, 100000)
pz = numpy.random.normal(0, 300, 100000)

**Computing one event at a time ("normal"):**

In [6]:
%%timeit
p = numpy.empty(100000)
for i in range(len(p)):                                   # for each px[i], py[i], pz[i]
    p[i] = numpy.sqrt(px[i]**2 + py[i]**2 + pz[i]**2)     # compute p[i]

202 ms ± 627 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


**Computing one column at a time ("sideways"):**

In [7]:
%%timeit
p = numpy.sqrt(px**2 + py**2 + pz**2)       # compute all px**2, then all py**2, then all pz**2, then sum all, then sqrt all

515 µs ± 39.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Normal math functions are *scalar* (e.g. binary operators like `+` or functions from `import math`). They perform one operation for each appearance in Python source code.

Numpy math functions are *vectorized.* Given equal-length arrays as input, they return the same length array as output, performing all loops in compiled C code, possibly doing 4 or 8 at a time in the CPU.

In [8]:
small_array = numpy.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [9]:
small_array**2

array([ 0,  1,  4,  9, 16, 25, 36, 49, 64, 81])

In [10]:
numpy.sqrt(small_array)

array([0.        , 1.        , 1.41421356, 1.73205081, 2.        ,
       2.23606798, 2.44948974, 2.64575131, 2.82842712, 3.        ])

In [11]:
# this won't work because math.sqrt wants a scalar number
import math
math.sqrt(small_array)

TypeError: only size-1 arrays can be converted to Python scalars

Numpy arrays are contiguous blocks of bytes in memory, just like C arrays.

In [16]:
small_array.view(numpy.float16)        # view (reinterpret_cast) the 64-bit integers as unsigned 8-bit integers

array([0.0e+00, 0.0e+00, 0.0e+00, 0.0e+00, 6.0e-08, 0.0e+00, 0.0e+00,
       0.0e+00, 1.2e-07, 0.0e+00, 0.0e+00, 0.0e+00, 1.8e-07, 0.0e+00,
       0.0e+00, 0.0e+00, 2.4e-07, 0.0e+00, 0.0e+00, 0.0e+00, 3.0e-07,
       0.0e+00, 0.0e+00, 0.0e+00, 3.6e-07, 0.0e+00, 0.0e+00, 0.0e+00,
       4.2e-07, 0.0e+00, 0.0e+00, 0.0e+00, 4.8e-07, 0.0e+00, 0.0e+00,
       0.0e+00, 5.4e-07, 0.0e+00, 0.0e+00, 0.0e+00], dtype=float16)

They can be multidimensional.

In [21]:
twod = small_array.reshape(2, 1, 5, 1, 2)     # view as 2 arrays of 5 elements each (a constant-time reinterpretation)
twod

ValueError: cannot reshape array of size 10 into shape (2,1,5,1,2)

They can even be arrays of _structs!_

In [28]:
table = numpy.array([(0, -9, 0.0), (1, -7, 1.1), (2, -5, 2.2), (3, -3, 3.3), (4, -1, 4.4), (5, 1, 5.5), (6, 3, 6.6), (7, 5, 7.7), (8, 7, 8.8), (9, 9, 9.9)],
                    dtype=[("one", numpy.uint8), ("two", numpy.int64), ("three", numpy.double)])
table["three"]

array([0. , 1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])

Besides being a common, agreed-upon language for sharing arrays with C/C++ and Fortran code, Numpy has powerful index selection:

In [35]:
small_array[-1:1:-2]

array([9, 7, 5, 3])

In [36]:
twod[1, ::2]

array([[[[5]],

        [[6]],

        [[7]],

        [[8]],

        [[9]]]])

In [38]:
table["three"]

array([0. , 1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7, 8.8, 9.9])

In [41]:
table["three"][[False, False, True, False, False, True, False, True, False, True, True]]

IndexError: boolean index did not match indexed array along dimension 0; dimension is 10 but corresponding boolean dimension is 11

In [44]:
table["three"][[7, 5, 5, 3, 2, 7, -2]]

array([7.7, 5.5, 5.5, 3.3, 2.2, 7.7, 8.8])

In [46]:
table["two"][table["two"]]

array([-7, -3,  1,  5,  9, -7, -3,  1,  5,  9])

In [48]:
table["three"][numpy.tile([7, 2], 5)]

array([7.7, 2.2, 7.7, 2.2, 7.7, 2.2, 7.7, 2.2, 7.7, 2.2])

The same rules apply to *assigning* to arrays.

In [49]:
small_array[[False, False, False, False, False, True, False, True, False, True]] = 5000, 7000, 9000

In [50]:
small_array

array([   0,    1,    2,    3,    4, 5000,    6, 7000,    8, 9000])

Be careful changing things in place!

In [51]:
view = small_array[5:]
view

array([5000,    6, 7000,    8, 9000])

In [52]:
view[0] = 999

In [53]:
small_array

array([   0,    1,    2,    3,    4,  999,    6, 7000,    8, 9000])

In [55]:
view.base is small_array

True

In [56]:
small_array.base is None

True

**Exercise:**

Suppose you're given a zillion `(px, py, pz, E)` 4-vectors and you want `(E, px, py, pz)` 4-vectors. Do it *fast!*

In [57]:
ZILLION = 1000000
fourvectors = numpy.empty((ZILLION, 4))
fourvectors[:, 0] = numpy.random.normal(0, 1, ZILLION)
fourvectors[:, 1] = numpy.random.normal(0, 1, ZILLION)
fourvectors[:, 2] = numpy.random.normal(0, 10, ZILLION)
fourvectors[:, 3] = numpy.random.normal(0, 10, ZILLION)**2
fourvectors[0]

array([-2.30413018e-01, -1.47658910e-01,  1.70037831e+00,  2.27222191e+02])

In [70]:
%%timeit
reordered = numpy.empty((ZILLION, 4))
reordered[:, 0] = fourvectors[:, 3]
reordered[:, 1] = fourvectors[:, 0]
reordered[:, 2] = fourvectors[:, 1]
reordered[:, 3] = fourvectors[:, 2]

21.5 ms ± 186 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# uproot: ROOT → Numpy

There are several ways to get ROOT data into Numpy arrays:

   * Pythonic iteration in PyROOT (super slow)
   * use ROOT's new `TTree::AsMatrix` (for flat tables)
   * through a custom C++ function
   * with `root_numpy` (compiles against a ROOT version; can segfault with version mismatch)
   * uproot* is a pure-Python, minimal dependency reimplementation of ROOT I/O

<i>(*disclosure: I'm the author)</i>

In [ ]:
!pip install uproot --force-reinstall

In [71]:
import uproot

In [72]:
events = uproot.open("http://scikit-hep.org/uproot/examples/HZZ.root")["events"]
events.keys()

[b'NJet',
 b'Jet_Px',
 b'Jet_Py',
 b'Jet_Pz',
 b'Jet_E',
 b'Jet_btag',
 b'Jet_ID',
 b'NMuon',
 b'Muon_Px',
 b'Muon_Py',
 b'Muon_Pz',
 b'Muon_E',
 b'Muon_Charge',
 b'Muon_Iso',
 b'NElectron',
 b'Electron_Px',
 b'Electron_Py',
 b'Electron_Pz',
 b'Electron_E',
 b'Electron_Charge',
 b'Electron_Iso',
 b'NPhoton',
 b'Photon_Px',
 b'Photon_Py',
 b'Photon_Pz',
 b'Photon_E',
 b'Photon_Iso',
 b'MET_px',
 b'MET_py',
 b'MChadronicBottom_px',
 b'MChadronicBottom_py',
 b'MChadronicBottom_pz',
 b'MCleptonicBottom_px',
 b'MCleptonicBottom_py',
 b'MCleptonicBottom_pz',
 b'MChadronicWDecayQuark_px',
 b'MChadronicWDecayQuark_py',
 b'MChadronicWDecayQuark_pz',
 b'MChadronicWDecayQuarkBar_px',
 b'MChadronicWDecayQuarkBar_py',
 b'MChadronicWDecayQuarkBar_pz',
 b'MClepton_px',
 b'MClepton_py',
 b'MClepton_pz',
 b'MCleptonPDGid',
 b'MCneutrino_px',
 b'MCneutrino_py',
 b'MCneutrino_pz',
 b'NPrimaryVertices',
 b'triggerIsoMu24',
 b'EventWeight']

One-per-event ROOT branches become Numpy arrays:

In [83]:
events.array("MET_px")

array([  5.912771,  24.765203, -25.785088, ...,  18.101646,  79.87519 ,
        19.713749], dtype=float32)

Multi-per-event ROOT branches become so-called "jagged arrays":

In [86]:
events.array("Jet_Px")

jaggedarray([[],
             [-38.874714],
             [],
             ...,
             [-3.7148185],
             [-36.361286 -15.256871],
             []])

A jagged array represents a list (events) of lists (particles).

In [87]:
px_events, py_events, pz_events = events.arrays(["Jet_Px", "Jet_Py", "Jet_Pz"], outputtype=tuple)

In [88]:
%%timeit
p = numpy.empty(2773)
j = 0
for i in range(events.numentries):
    for px, py, pz in zip(px_events[i], py_events[i], pz_events[i]):
        p[j] = math.sqrt(px**2 + py**2 + pz**2)
        j += 1

36.7 ms ± 3.56 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


But it consists of three contiguous arrays.

In [89]:
print(px_events.starts)                # content index where each event starts (inclusive)
print(px_events.stops)                 # content index where each event stops (exclusive)
print(px_events.content)               # content data

[   0    0    1 ... 2770 2771 2773]
[   0    1    1 ... 2771 2773 2773]
[-38.874714  -71.69521    36.60637   ...  -3.7148185 -36.361286
 -15.256871 ]


In [90]:
%%timeit
p = numpy.sqrt(px_events.content**2 + py_events.content**2 + pz_events.content**2)

9.17 µs ± 965 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


Numpy is designed for flat arrays, so X of all particles → Y of all particles is easy. What about particle attributes *per event?*

In [ ]:
p = numpy.full(events.numentries, numpy.nan)    # initialize with nan
for i in range(events.numentries):
    for px, py, pz in zip(px_events[i], py_events[i], pz_events[i]):
        # only fill first in each event
        if numpy.isnan(p[i]):
            p[i] = math.sqrt(px**2 + py**2 + pz**2)
p

**Step 1:** Which events have at least one jet?

In [ ]:
hasajet = (px_events.stops - px_events.starts != 0)
hasajet

**Step 2:** Get indexes of first jets in events with at least one jet.

In [ ]:
firsts = px_events.starts[hasajet]
firsts

**Step 3:** Gather values from per-particle arrays into per-event arrays.

In [ ]:
px, py, pz = px_events.content[firsts], py_events.content[firsts], pz_events.content[firsts]

**Step 4:** Vectorized calculation, assigned to events with at least one jet.

In [ ]:
p = numpy.full(events.numentries, numpy.nan)       # initialize with nan
p[hasajet] = numpy.sqrt(px**2 + py**2 + pz**2)     # assign to events through the mask
p

**Exercise:**

Compute the invariant mass of the first two muons in each event that has at least two muons.

$m = \sqrt{(E_1 + E_2)^2 - (px_1 + px_2)^2 - (py_1 + py_2)^2 - (pz_1 + pz_2)^2}$

In [ ]:
q_events = events.array("Muon_Charge")
starts, stops = q_events.starts, q_events.stops
energy = events.array("Muon_E").content
px = events.array("Muon_Px").content
py = events.array("Muon_Py").content
pz = events.array("Muon_Pz").content

In [ ]:
mass = ???

In [ ]:
!pip install histbook --user

In [ ]:
from histbook import Hist, bin
# from vega import VegaLite                                             # if Jupyter Notebook, not JupyterLab

In [ ]:
Hist(bin("mass", 200, 0, 200), fill=mass).step(width=400, height=400)   # .to(VegaLite)

![](src/visualization.png)

# Pandas: data analysis on indexed tables

Pandas is a library for manipulating tabular data, but as we've just seen, so is Numpy.

Pandas is based on Numpy. So what's its "value added?"

## Rich indexing!!!

Although the elements of a Numpy array can be accessed in fancy ways, each item is conceptually addressed by a single integer: the entry number. Indexes in Pandas can be typed (e.g. string keys like a `dict`), structured like the two-component index for jagged data below, and sparse (as below).

In [ ]:
df = events.pandas.df(["MET*", "Muon*"])
df

In [ ]:
df.loc[2]

Indexes can be fluidly manipulated. For instance, convert this structured index into integer-indexed columns:

In [ ]:
df.reset_index()

Flatten the sparse subentries into dense columns: a column for each number of muons:

In [ ]:
df2 = df.pivot_table(index="entry", columns="subentry")
df2

In [ ]:
df2[b"Muon_E"]

Numpy concepts and functions apply to Pandas.

In [ ]:
# make a mask of events with at least two muons (second muon index is not nan)
has2muons = numpy.logical_not(numpy.isnan(df2[b"Muon_E", 1]))
df3 = df2[has2muons]

# select particle attributes for the first and second muons through filter
e1,  e2  = df3[b"Muon_E",  0], df3[b"Muon_E",  1]
px1, px2 = df3[b"Muon_Px", 0], df3[b"Muon_Px", 1]
py1, py2 = df3[b"Muon_Py", 0], df3[b"Muon_Py", 1]
pz1, pz2 = df3[b"Muon_Pz", 0], df3[b"Muon_Pz", 1]

# vectorized calculation like Numpy
mass = numpy.sqrt((e1 + e2)**2 - (px1 + px2)**2 - (py1 + py2)**2 - (pz1 + pz2)**2)
mass

In [ ]:
%matplotlib inline
mass.hist()

# Faster math: NumExpr, Numba, and Cython

The Python development strategy is to write slow Python until it gets to be a problem and then accelerate the problem spots.

In [ ]:
# some require actual Numpy arrays, not Pandas series
e1, e2, px1, px2, py1, py2, pz1, pz2 = (
    numpy.array(e1), numpy.array(e2), numpy.array(px1), numpy.array(px2),
    numpy.array(py1), numpy.array(py2), numpy.array(pz1), numpy.array(pz2))

## NumExpr

Mini-compiler to process Numpy arrays in one pass, rather than one per column.

In [ ]:
import numexpr
numexpr.evaluate("sqrt((e1 + e2)**2 - (px1 + px2)**2 - (py1 + py2)**2 - (pz1 + pz2)**2)")

## Numba

More general: uses LLVM to just-in-time compile Python code to native bytecode.

In [ ]:
!pip install numba --user

In [ ]:
import numba

@numba.jit
def f(e1, e2, px1, px2, py1, py2, pz1, pz2):
    out = numpy.empty(len(e1))
    for i in range(len(e1)):
        out[i] = math.sqrt((e1[i] + e2[i])**2 - (px1[i] + px2[i])**2 - (py1[i] + py2[i])**2 - (pz1[i] + pz2[i])**2)
    return out

f(e1, e2, px1, px2, py1, py2, pz1, pz2)

## Cython

Cython is a half-way language between C++ and Python for static compilation and C++ bindings.

In [ ]:
%load_ext Cython

*(continued next page)*

In [ ]:
%%cython -a --cplus -c-std=c++11 -c-O3

import numpy

cdef extern from *:
    """
    void doit(int N, float* out, float* e1, float* e2, float* px1, float* px2,
                                 float* py1, float* py2, float* pz1, float* pz2) {
        for (int i = 0;  i < N;  i++) {
            out[i] = sqrt((e1[i] + e2[i])*(e1[i] + e2[i]) - (px1[i] + px2[i])*(px1[i] + px2[i]) -
                          (py1[i] + py2[i])*(py1[i] + py2[i]) - (pz1[i] + pz2[i])*(pz1[i] + pz2[i]));
        }
    }
    """
    void doit(int N, float* out, float* e1, float* e2, float* px1, float* px2, float* py1, float* py2, float* pz1, float* pz2)

def g(e1, e2, px1, px2, py1, py2, pz1, pz2):
    if (not isinstance(e1, numpy.ndarray) or not isinstance(e2, numpy.ndarray) or
        not isinstance(px1, numpy.ndarray) or not isinstance(px2, numpy.ndarray) or
        not isinstance(py1, numpy.ndarray) or not isinstance(py2, numpy.ndarray) or
        not isinstance(pz1, numpy.ndarray) or not isinstance(pz2, numpy.ndarray)):
        raise TypeError("these are not arrays")

    out = numpy.empty(len(e1), dtype=numpy.float32)    
    doit(len(e1), <float*>(<size_t>out.ctypes.data),
         <float*>(<size_t>e1.ctypes.data), <float*>(<size_t>e2.ctypes.data),
         <float*>(<size_t>px1.ctypes.data), <float*>(<size_t>px2.ctypes.data),
         <float*>(<size_t>py1.ctypes.data), <float*>(<size_t>py2.ctypes.data),
         <float*>(<size_t>pz1.ctypes.data), <float*>(<size_t>pz2.ctypes.data))
    return out

In [ ]:
g(e1, e2, px1, px2, py1, py2, pz1, pz2)

# Dask: delayed and distributed computing

In [ ]:
!pip install "dask[complete]" --user

In [ ]:
lazyarrays = uproot.daskarrays("http://scikit-hep.org/uproot/examples/HZZ.root", "events")

In [ ]:
lazyarrays

In [ ]:
pt = numpy.sqrt(lazyarrays[b"MET_px"]**2 + lazyarrays[b"MET_py"]**2)
pt

`pt` is an *instruction* for a calculation that can be distributed across a cluster. (We don't have a cluster.)

In [ ]:
pt.compute()

# NumPythia, PyJet, and PyPDT: for quick generator-level studies

## NumPythia

In [ ]:
!pip install numpythia --user      # it takes a long time to compile Pythia...

In [ ]:
import numpythia
import numpythia.testcmnd
pythia = numpythia.Pythia(numpythia.testcmnd.get_cmnd("w"), random_state=1)

In [ ]:
import pandas
events = list(pythia(events=1))    # pythia is a generator; evaluate it
pandas.DataFrame(events[0].all())  # event.all(SELECTION) returns a Numpy record array, so of course we Pandas it

## pyjet

In [ ]:
!pip install pyjet --user

In [ ]:
import pyjet

fourvectors = events[0].all()[["E", "px", "py", "pz"]]
for jet in pyjet.cluster(fourvectors, R=1.0, p=-1, ep=True).inclusive_jets():
    print(jet)

## PyPDT

In [ ]:
!pip install pypdt --user

In [ ]:
import pypdt

for pdgid in set(events[0].all()["pdgid"]):
    p = pypdt.get(pdgid)
    if p is not None:
        print("{}: {} mass: {} width {} lifetime {} ctau {}".format(
            pdgid, p.name, p.mass, p.width, p.lifetime, p.ctau))

# The End!